In [1]:
import numpy as np
import rasterio
import tensorflow as tf
import os
from rasterio.mask import mask
import geopandas as gpd
from tensorflow.keras import layers, models
from tensorflow.keras import mixed_precision

# Set float32 for broader compatibility
mixed_precision.set_global_policy('float32')

# --------------------
# Mapping Dictionary
# --------------------
glclu_to_simplified = {
    0: None, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1,
    104: 4, 107: 4, 108: 4,
    100: 3, 112: 3, 111: 3,
    125: 2, 126: 2, 127: 2, 128: 2, 129: 2, 130: 2, 131: 2, 132: 2,
    200: 5, 201: 5, 202: 5, 204: 5, 205: 5, 207: 5,
    244: 9, 250: 7,
    113: 6, 114: 6,
    140: 8, 141: 8, 142: 8, 143: 8, 144: 8
}

# --------------------
# Utilities
# --------------------
def load_and_remap_tif(path, mapping_dict, downsample=2):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(np.uint16)
        if downsample > 1:
            arr = arr[::downsample, ::downsample]
        remapped = np.vectorize(mapping_dict.get)(arr).astype('float32')
        return np.nan_to_num(remapped, nan=0).astype(np.uint8)

def extract_3d_patches(sequence, patch_size=(32, 32), stride=(16, 16)):
    patches = []
    h, w = sequence[0].shape
    for i in range(0, h - patch_size[0] + 1, stride[0]):
        for j in range(0, w - patch_size[1] + 1, stride[1]):
            cube = [img[i:i+patch_size[0], j:j+patch_size[1]] for img in sequence]
            cube = np.stack(cube, axis=0)
            patches.append(cube)
    return np.expand_dims(np.array(patches), axis=-1)

def reconstruct(patches, shape, patch_size=(32, 32), stride=(16, 16)):
    h, w = shape
    result = np.zeros((h, w))
    count = np.zeros((h, w))
    idx = 0
    for i in range(0, h - patch_size[0] + 1, stride[0]):
        for j in range(0, w - patch_size[1] + 1, stride[1]):
            result[i:i+patch_size[0], j:j+patch_size[1]] += patches[idx]
            count[i:i+patch_size[0], j:j+patch_size[1]] += 1
            idx += 1
    return (result / count).round().astype(np.uint8)

def save_tif(img, ref_path, out_path):
    with rasterio.open(ref_path) as src:
        meta = src.meta.copy()
        meta.update(dtype=rasterio.uint8, count=1)
    with rasterio.open(out_path, 'w', **meta) as dst:
        dst.write(img.astype(np.uint8), 1)

# --------------------
# Load Model
# --------------------
model = tf.keras.models.load_model("LandCoverModel_Optimized")
print("✅ Model Loaded")

# --------------------
# Load Training Data
# --------------------
years = [2000, 2005, 2010, 2015, 2020]
file_paths = [f"Covai_LandCover_{year}.tif" for year in years]
data = [load_and_remap_tif(fp, glclu_to_simplified, downsample=2) for fp in file_paths]

# Class Mapping
all_unique = set()
for img in data:
    all_unique.update(np.unique(img))
unique_classes = sorted(all_unique)
class_map = {v: i for i, v in enumerate(unique_classes)}
inverse_class_map = {i: v for v, i in class_map.items()}

data_indexed = [np.vectorize(class_map.get)(img) for img in data]

# --------------------
# Prediction Function
# --------------------
def predict_next(sequence, shape):
    X = extract_3d_patches(sequence)
    preds = model.predict(X, batch_size=2, verbose=1)
    preds = np.argmax(preds, axis=-1)
    return reconstruct(preds, shape)

# --------------------
# Predict Future Years
# --------------------
shape = data[0].shape
pred_2025 = predict_next(data_indexed[1:], shape)
pred_2030 = predict_next([data_indexed[2], data_indexed[3], data_indexed[4], pred_2025], shape)
pred_2035 = predict_next([data_indexed[3], data_indexed[4], pred_2025, pred_2030], shape)
pred_2040 = predict_next([data_indexed[4], pred_2025, pred_2030, pred_2035], shape)

# Map predictions back to land cover codes
pred_2025_mapped = np.vectorize(inverse_class_map.get)(pred_2025)
pred_2030_mapped = np.vectorize(inverse_class_map.get)(pred_2030)
pred_2035_mapped = np.vectorize(inverse_class_map.get)(pred_2035)
pred_2040_mapped = np.vectorize(inverse_class_map.get)(pred_2040)

# --------------------
# Save All as .tif
# --------------------
ref_tif = file_paths[-1]  # Use 2020 as template
save_tif(pred_2025_mapped, ref_tif, "Simplified_Predicted_Simplified_2025.tif")
save_tif(pred_2030_mapped, ref_tif, "Simplified_Predicted_Simplified_2030.tif")
save_tif(pred_2035_mapped, ref_tif, "Simplified_Predicted_Simplified_2035.tif")
save_tif(pred_2040_mapped, ref_tif, "Simplified_Predicted_Simplified_2040.tif")

print("📁 All predictions saved in same format as actual data.")

/Users/pratiksha/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


✅ Model Loaded
6528/6528 [==============================] - 81s 12ms/step


/var/folders/nk/px32ntw90239_n34ks4_61cc0000gn/T/ipykernel_10938/1000111115.py:58: RuntimeWarning: invalid value encountered in divide
  return (result / count).round().astype(np.uint8)
/var/folders/nk/px32ntw90239_n34ks4_61cc0000gn/T/ipykernel_10938/1000111115.py:58: RuntimeWarning: invalid value encountered in cast
  return (result / count).round().astype(np.uint8)


6528/6528 [==============================] - 86s 13ms/step
📁 All predictions saved in same format as actual data.


In [9]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import os
from collections import defaultdict

# --- Load Shapefile ---
import os
shapefile_path = "./CBE_Taluk/CBE_TALUKs.shp"

gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs("EPSG:4326")  # Ensure same CRS

# --- Define .tif files in order ---
tif_files = {
    "2000": "Simplified_Covai_LandCover_2000.tif",
    "2005": "Simplified_Covai_LandCover_2005.tif",
    "2010": "Simplified_Covai_LandCover_2010.tif",
    "2015": "Simplified_Covai_LandCover_2015.tif",
    "2020": "Simplified_Covai_LandCover_2020.tif",
    "2025": "Predicted_Simplified_2025.tif",
    "2030": "Predicted_Simplified_2030.tif",
    "2035": "Predicted_Simplified_2035.tif",
    "2040": "Predicted_Simplified_2040.tif",
}

# Optional: mapping class numbers to labels
class_labels = {
    1: "Tree Cover", 2: "Shrubland", 3: "Grassland", 4: "Cropland",
    5: "Built-up", 6: "Barren", 7: "Water", 8: "Snow/Ice", 9: "Wetland"
}

# --- Loop Through Each Year & Taluk ---
for year, tif_path in tif_files.items():
    print(f"\n🟨 Year: {year} | File: {tif_path}")

    try:
        with rasterio.open(tif_path) as src:
            for i, row in gdf.iterrows():
                taluk_name = row['NAME_3'] if 'NAME_3' in row else row['NAME']  # Adjust if different
                geom = [row["geometry"]]

                try:
                    out_image, out_transform = mask(src, geom, crop=True)
                    masked = out_image[0]
                    unique, counts = np.unique(masked[masked > 0], return_counts=True)
                    result = dict(zip(unique, counts))

                    print(f"📌 {taluk_name}:")
                    for cls, count in result.items():
                        label = class_labels.get(cls, f"Class {cls}")
                        print(f"   - {label} (class {cls}): {count} pixels")

                except Exception as e:
                    print(f"❌ Could not mask {taluk_name}: {e}")

    except Exception as e:
        print(f"❌ Failed to open {tif_path}: {e}")



🟨 Year: 2000 | File: Simplified_Covai_LandCover_2000.tif
📌 Avanashi:
   - Tree Cover (class 1): 21394 pixels
   - Shrubland (class 2): 109 pixels
   - Grassland (class 3): 17 pixels
   - Built-up (class 5): 158 pixels
   - Water (class 7): 115942 pixels
   - Wetland (class 9): 342203 pixels
📌 Coimbatore:
   - Tree Cover (class 1): 24023 pixels
   - Shrubland (class 2): 1394 pixels
   - Built-up (class 5): 2971 pixels
   - Water (class 7): 324260 pixels
   - Snow/Ice (class 8): 11 pixels
   - Wetland (class 9): 333843 pixels
📌 Mettuppalaiyam:
   - Tree Cover (class 1): 2612 pixels
   - Shrubland (class 2): 321 pixels
   - Grassland (class 3): 5 pixels
   - Cropland (class 4): 2 pixels
   - Built-up (class 5): 3192 pixels
   - Water (class 7): 56618 pixels
   - Snow/Ice (class 8): 41 pixels
   - Wetland (class 9): 189141 pixels
📌 Pollachi:
   - Tree Cover (class 1): 1208 pixels
   - Shrubland (class 2): 5498 pixels
   - Grassland (class 3): 60 pixels
   - Cropland (class 4): 10 pixels
 

In [19]:
import rasterio

with rasterio.open("Simplified_Covai_LandCover_2000.tif") as src:
    print("CRS:", src.crs)
    print("Transform:", src.transform)
    print("Pixel size (width x height):", src.res)
    print("Bounds:", src.bounds)


CRS: EPSG:4326
Transform: | 0.00, 0.00, 76.65|
| 0.00,-0.00, 11.40|
| 0.00, 0.00, 1.00|
Pixel size (width x height): (0.00026949458523585647, 0.00026949458523585647)
Bounds: BoundingBox(left=76.6472393684927, bottom=10.208448272299776, right=77.4904879256957, top=11.396919393189904)


In [21]:
from rasterio.io import MemoryFile

# Inside the loop where you already calculated `temp_raster`, `transform`, etc.
with MemoryFile() as memfile:
    with memfile.open(
        driver="GTiff",
        height=height,
        width=width,
        count=1,
        dtype=temp_raster.dtype,
        crs=dst_crs,
        transform=transform,
    ) as dataset:
        dataset.write(temp_raster[0], 1)

        for i, row in gdf_utm.iterrows():
            taluk_name = row.get("NAME_3") or row.get("NAME") or f"Taluk_{i}"
            geom = [row.geometry]

            try:
                out_image, _ = mask(dataset, geom, crop=True, all_touched=True)
                masked = out_image[0]
                unique, counts = np.unique(masked[masked > 0], return_counts=True)

                for cls, count in zip(unique, counts):
                    land_type = class_labels.get(cls, f"Class_{cls}")
                    area_ha = count * pixel_area_ha
                    results[land_type][taluk_name][year] = area_ha

            except Exception as e:
                print(f"⚠️ No overlap or mask fail for {taluk_name}: {e}")


In [22]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
import numpy as np
from collections import defaultdict

# --- Load Taluk Shapefile and Convert to UTM ---
shapefile_path = "./CBE_Taluk/CBE_TALUKs.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs("EPSG:4326")  # Match initial .tif CRS

# --- Convert Shapefile to UTM for accurate area computation ---
utm_crs = "EPSG:32643"  # UTM zone 43N (suitable for Tamil Nadu region)
gdf_utm = gdf.to_crs(utm_crs)

# --- Define Land Cover .tif Files ---
tif_files = {
    "2000": "Simplified_Covai_LandCover_2000.tif",
    "2005": "Simplified_Covai_LandCover_2005.tif",
    "2010": "Simplified_Covai_LandCover_2010.tif",
    "2015": "Simplified_Covai_LandCover_2015.tif",
    "2020": "Simplified_Covai_LandCover_2020.tif",
    "2025": "Predicted_Simplified_2025.tif",
    "2030": "Predicted_Simplified_2030.tif",
    "2035": "Predicted_Simplified_2035.tif",
    "2040": "Predicted_Simplified_2040.tif",
}

# --- Class Labels ---
class_labels = {
    1: "Tree Cover", 2: "Shrubland", 3: "Grassland", 4: "Cropland",
    5: "Built-up", 6: "Barren", 7: "Water", 8: "Snow/Ice", 9: "Wetland"
}

# --- Results Dictionary ---
results = defaultdict(lambda: defaultdict(dict))

# --- Loop Through Each Year ---
for year, tif_path in tif_files.items():
    print(f"\n🟨 Processing Year: {year} | File: {tif_path}")

    try:
        with rasterio.open(tif_path) as src:
            src_crs = src.crs
            transform, width, height = calculate_default_transform(
                src.crs, utm_crs, src.width, src.height, *src.bounds
            )

            # Reproject the raster to UTM
            temp_raster = np.empty((1, height, width), dtype=src.dtypes[0])
            reproject(
                source=rasterio.band(src, 1),
                destination=temp_raster,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=utm_crs,
                resampling=Resampling.nearest
            )

            # Calculate pixel area in hectares (1 ha = 10,000 m²)
            pixel_width = transform[0]
            pixel_height = -transform[4]
            pixel_area_ha = pixel_width * pixel_height / 10000
            print(f"📏 Pixel area: {pixel_width * pixel_height:.2f} m² ≈ {pixel_area_ha:.4f} ha")

            # Write reprojected raster to memory and apply mask
            with MemoryFile() as memfile:
                with memfile.open(
                    driver="GTiff",
                    height=height,
                    width=width,
                    count=1,
                    dtype=temp_raster.dtype,
                    crs=utm_crs,
                    transform=transform,
                ) as dataset:
                    dataset.write(temp_raster[0], 1)

                    for i, row in gdf_utm.iterrows():
                        taluk_name = row.get("NAME_3") or row.get("NAME") or f"Taluk_{i}"
                        geom = [row.geometry]

                        try:
                            out_image, _ = mask(dataset, geom, crop=True, all_touched=True)
                            masked = out_image[0]
                            unique, counts = np.unique(masked[masked > 0], return_counts=True)

                            if len(unique) == 0:
                                print(f"⚠️ No data for {taluk_name}")
                                continue

                            print(f"📌 {taluk_name} ({year}):")
                            for cls, count in zip(unique, counts):
                                land_type = class_labels.get(cls, f"Class_{cls}")
                                area_ha = count * pixel_area_ha
                                results[land_type][taluk_name][year] = area_ha
                                print(f"   - {land_type} ({cls}): {area_ha:.2f} ha")

                        except Exception as e:
                            print(f"⚠️ No overlap or mask fail for {taluk_name}: {e}")
    except Exception as e:
        print(f"❌ Failed to process {tif_path}: {e}")



🟨 Processing Year: 2000 | File: Simplified_Covai_LandCover_2000.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha
📌 Avanashi (2000):
   - Tree Cover (1): 1899.83 ha
   - Shrubland (2): 9.97 ha
   - Grassland (3): 1.50 ha
   - Built-up (5): 13.85 ha
   - Water (7): 10219.99 ha
   - Wetland (9): 30196.94 ha
📌 Coimbatore (2000):
   - Tree Cover (1): 2121.64 ha
   - Shrubland (2): 122.64 ha
   - Built-up (5): 260.90 ha
   - Water (7): 28495.81 ha
   - Snow/Ice (8): 0.97 ha
   - Wetland (9): 29403.90 ha
📌 Mettuppalaiyam (2000):
   - Tree Cover (1): 231.96 ha
   - Shrubland (2): 27.97 ha
   - Grassland (3): 0.44 ha
   - Cropland (4): 0.18 ha
   - Built-up (5): 280.76 ha
   - Water (7): 4980.74 ha
   - Snow/Ice (8): 3.71 ha
   - Wetland (9): 16672.54 ha
📌 Pollachi (2000):
   - Tree Cover (1): 106.67 ha
   - Shrubland (2): 483.25 ha
   - Grassland (3): 5.29 ha
   - Cropland (4): 0.88 ha
   - Built-up (5): 1895.15 ha
   - Water (7): 12292.67 ha
   - Snow/Ice (8): 5.91 ha
   - Wetland (9): 24994.03 ha
📌 

In [23]:
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterio.io import MemoryFile
import numpy as np
import pandas as pd
from collections import defaultdict
import re

# --- Load Taluk Shapefile and Convert to UTM ---
shapefile_path = "./CBE_Taluk/CBE_TALUKs.shp"
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs("EPSG:4326")

utm_crs = "EPSG:32643"  # UTM for Tamil Nadu
gdf_utm = gdf.to_crs(utm_crs)

# --- Define Land Cover .tif Files ---
tif_files = {
    "2000": "Simplified_Covai_LandCover_2000.tif",
    "2005": "Simplified_Covai_LandCover_2005.tif",
    "2010": "Simplified_Covai_LandCover_2010.tif",
    "2015": "Simplified_Covai_LandCover_2015.tif",
    "2020": "Simplified_Covai_LandCover_2020.tif",
    "2025": "Predicted_Simplified_2025.tif",
    "2030": "Predicted_Simplified_2030.tif",
    "2035": "Predicted_Simplified_2035.tif",
    "2040": "Predicted_Simplified_2040.tif",
}

# --- Land Cover Class Labels ---
class_labels = {
    1: "Tree Cover", 2: "Shrubland", 3: "Grassland", 4: "Cropland",
    5: "Built-up", 6: "Barren", 7: "Water", 8: "Snow/Ice", 9: "Wetland"
}

# --- Output Container ---
results = defaultdict(lambda: defaultdict(dict))  # {class -> {taluk -> {year -> hectares}}}

# --- Loop Over All Years ---
for year, tif_path in tif_files.items():
    print(f"\n🟨 Processing Year: {year} | File: {tif_path}")
    try:
        with rasterio.open(tif_path) as src:
            # Reproject raster to UTM for accurate area calc
            transform, width, height = calculate_default_transform(
                src.crs, utm_crs, src.width, src.height, *src.bounds
            )

            temp_raster = np.empty((1, height, width), dtype=src.dtypes[0])
            reproject(
                source=rasterio.band(src, 1),
                destination=temp_raster,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=transform,
                dst_crs=utm_crs,
                resampling=Resampling.nearest
            )

            pixel_width = transform[0]
            pixel_height = -transform[4]
            pixel_area_ha = pixel_width * pixel_height / 10000
            print(f"📏 Pixel area: {pixel_width * pixel_height:.2f} m² ≈ {pixel_area_ha:.4f} ha")

            # Use memory file to apply mask on reprojected raster
            with MemoryFile() as memfile:
                with memfile.open(
                    driver="GTiff",
                    height=height,
                    width=width,
                    count=1,
                    dtype=temp_raster.dtype,
                    crs=utm_crs,
                    transform=transform,
                ) as dataset:
                    dataset.write(temp_raster[0], 1)

                    for i, row in gdf_utm.iterrows():
                        taluk_name = row.get("NAME_3") or row.get("NAME") or f"Taluk_{i}"
                        geom = [row.geometry]

                        try:
                            out_image, _ = mask(dataset, geom, crop=True, all_touched=True)
                            masked = out_image[0]
                            unique, counts = np.unique(masked[masked > 0], return_counts=True)

                            for cls, count in zip(unique, counts):
                                land_type = class_labels.get(cls, f"Class_{cls}")
                                area_ha = count * pixel_area_ha
                                results[land_type][taluk_name][year] = area_ha
                        except Exception as e:
                            print(f"⚠️ No overlap or mask fail for {taluk_name}: {e}")
    except Exception as e:
        print(f"❌ Failed to open {tif_path}: {e}")

# --- Save Results to Excel ---
def sanitize_sheet_name(name):
    return re.sub(r'[\\/*?:\[\]]', "_", name)[:31]

output_file = "LandCover_Talukwise_Hectares.xlsx"
with pd.ExcelWriter(output_file) as writer:
    for land_type, taluk_data in results.items():
        df = pd.DataFrame(taluk_data).T  # rows = taluks, columns = years
        df.index.name = "Taluk"
        df = df.reindex(sorted(df.columns), axis=1)  # sort years
        df.fillna(0, inplace=True)
        df = df.astype(float).round(2)

        if df.empty:
            continue

        safe_name = sanitize_sheet_name(land_type)
        df.to_excel(writer, sheet_name=safe_name)

print(f"\n✅ Saved to '{output_file}' in hectares ✅")



🟨 Processing Year: 2000 | File: Simplified_Covai_LandCover_2000.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2005 | File: Simplified_Covai_LandCover_2005.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2010 | File: Simplified_Covai_LandCover_2010.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2015 | File: Simplified_Covai_LandCover_2015.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2020 | File: Simplified_Covai_LandCover_2020.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2025 | File: Predicted_Simplified_2025.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2030 | File: Predicted_Simplified_2030.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2035 | File: Predicted_Simplified_2035.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

🟨 Processing Year: 2040 | File: Predicted_Simplified_2040.tif
📏 Pixel area: 882.33 m² ≈ 0.0882 ha

✅ Saved to 'LandCover_Talukwise_Hectares.xlsx' in hectares ✅
